In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 2.7.15+ (default, Nov 27 2018, 23:36:35) 
[GCC 7.3.0]
NLTK: 3.2.5
Scikit-learn: 0.20.3
Pandas: 0.24.2
Numpy: 1.14.1


In [2]:
# Carregando dados
import pandas as pd
import numpy as np

df = pd.read_csv('data/dataset_classes.txt', header=None, encoding='utf-8', sep='\t')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null object
1    793 non-null object
dtypes: object(2)
memory usage: 12.5+ KB
None
                  0                                                  1
0     mand seguranc  inexecuca contrat sanca afast reexam prov clau...
1     respons civil  acident aer decretol n dol eventual respons at...
2  processual civil  intimaca art cpc irregular nao comprov recurs ...
3     process civil  honorari execuca legitim part superior tribuna...
4         administr  servidor public artig lei n artig lei n revoga...


In [3]:
# verificação das classes dos acórdãos

classes = df[0]
print(classes.value_counts())

processual civil                             223
agrav regimental                             106
tributari                                     68
hab corpus                                    59
embarg declaraca                              48
process civil                                 47
administr                                     45
recurs especial                               25
previdenciari                                 13
recurs ordinari                               13
agrav intern                                   9
processual penal                               9
embarg declaratori                             8
penal                                          8
embarg divergenc                               7
conflit negat competenc                        6
process penal                                  5
direit administr                               5
processual                                     5
execuca fiscal                                 5
civil               

In [4]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes
encoder = LabelEncoder()
classes = encoder.fit_transform(classes)

print(classes[:10])
len(classes)

[51 75 65 62  4 48 58 55 80 79]


793

In [5]:
# criação da lista de ementas
ementas = df[1]
print(ementas[:10])

0    inexecuca contrat sanca afast reexam prov clau...
1    acident aer decretol n dol eventual respons at...
2    intimaca art cpc irregular nao comprov recurs ...
3    honorari execuca legitim part superior tribuna...
4    servidor public artig lei n artig lei n revoga...
5    iss arrend mercantil incidenc mat constitucion...
6    processual penal art codig process penal conhe...
7    recurs especial art incis art ambos codig pena...
8    praz prescricional lei complement n artig lei ...
9    licitaca aca anulator indefer tutel antecip ag...
Name: 1, dtype: object


In [6]:
# uso de word2vec para a extração de features

from gensim.models import Word2Vec

ementas_list = [word for word in ementas.iteritems()]
ementas_list_list = []
for doc in ementas_list:
    ementas_list_list.append(doc[1].split())

X = Word2Vec(ementas_list_list, min_count=2)
X.wv.init_sims() # para inicializar model.vw.syn0norm, necessário para o cálculo da média da
print(X)

Word2Vec(vocab=3410, size=100, alpha=0.025)


In [7]:
# funções para calcular a média vetorial das palavras de cada ementa

def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [8]:
# função para tokenizar as palavras com frequência maior que 2

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [9]:
# Like any other supervised machine learning problem, we need to divide data into 
# 20% test set and 80% training set

import gensim
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


train, test = train_test_split(df, test_size=0.2, random_state=0)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r[1]), axis=1).values

X_train = word_averaging_list(X.wv,train_tokenized)
X_test = word_averaging_list(X.wv,test_tokenized)

y_train = train[0]
y_test = test[0]

print(X_train)
print(X_test)

[[ 0.14434595  0.02304993  0.04989565 ... -0.07615176  0.02941636
  -0.04480303]
 [ 0.12452289  0.03159952  0.05815442 ... -0.06920306  0.03524349
  -0.02274381]
 [ 0.13089664  0.0446075   0.05487002 ... -0.0810603   0.03045238
  -0.0177536 ]
 ...
 [ 0.13796405  0.02995665  0.05505783 ... -0.07377195  0.02984725
  -0.04124976]
 [ 0.1385827   0.04720109  0.04975444 ... -0.07616398  0.04028198
  -0.03079266]
 [ 0.13406566  0.02936492  0.05563158 ... -0.07149168  0.03391809
  -0.03358595]]
[[ 0.13541737  0.04342233  0.05166049 ... -0.07384245  0.0387635
  -0.03114915]
 [ 0.1759237  -0.00786242  0.03587133 ... -0.09030756  0.02455668
  -0.06454533]
 [ 0.11141677  0.04794783  0.06108342 ... -0.06697238  0.03304423
  -0.01333338]
 ...
 [ 0.12622513  0.0354398   0.05465707 ... -0.07477427  0.03180372
  -0.01997102]
 [ 0.12316269  0.03934731  0.06462952 ... -0.05892854  0.04340352
  -0.03298957]
 [ 0.16833887 -0.00659573  0.0352239  ... -0.09245081  0.02598402
  -0.04273848]]


In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 33.3333333333
Decision Tree Accuracy: 20.7547169811
Random Forest Accuracy: 35.2201257862
Logistic Regression Accuracy: 30.8176100629
SGD Classifier Accuracy: 30.8176100629
Naive Bayes Accuracy: 30.8176100629


In [20]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(solver='lbfgs', multi_class='multinomial'),
    SGDClassifier(max_iter = 100, tol=1e-3),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


votingClassifier = VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1)
votingClassifier.fit(X_train, y_train)
y_pred = votingClassifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 36.4779874214


In [86]:
# relatório de classificação

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print((accuracy_score(y_test, y_pred)*100))

[[93  7]
 [46 13]]
              precision    recall  f1-score   support

       False       0.67      0.93      0.78       100
        True       0.65      0.22      0.33        59

   micro avg       0.67      0.67      0.67       159
   macro avg       0.66      0.58      0.55       159
weighted avg       0.66      0.67      0.61       159

66.66666666666666
